In [ ]:
import warnings
warnings.filterwarnings("ignore")
 
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
import numpy as np
#import tensorflow as tf
#from tf import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D, Flatten,BatchNormalization, Activation
from keras.optimizers import RMSprop , Adam ,SGD
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
#from callbacks import ModelCheckpoint, ReduceLROnPlateau
from IPython.display import Image 
import matplotlib.pyplot as plt
import os
import efficientnet.keras as enet

In [ ]:
from keras.backend import sigmoid


class SwishActivation(Activation):

    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'


def swish_act(x, beta=1):
    return (x * sigmoid(beta * x))


from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

get_custom_objects().update({'swish_act': SwishActivation(swish_act)})


In [ ]:
model = enet.EfficientNetB0(include_top=False, input_shape=(150,50,3), pooling='avg', weights='imagenet')
 
# Adding 2 fully-connected layers to B0.
x = model.output
 
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
 
x = Dense(512)(x)
x = BatchNormalization()(x)

x = Activation(swish_act)(x)
x = Dropout(0.5)(x)
 
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation(swish_act)(x)
 
x = Dense(64)(x)
 
x = Dense(32)(x)
 
x = Dense(16)(x)
 
# Output layer
predictions = Dense(1, activation="sigmoid")(x)
 
model_final = Model(inputs = model.input, outputs = predictions)
 
model_final.summary()

In [ ]:
model_final.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])
 
mcp_save = ModelCheckpoint('/gdrive/My Drive/EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_acc')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1,)
#print("Training....")
model_final.fit(x_train, y_train,
              batch_size=32,
              epochs=10,
              validation_split=0.1,
              callbacks=[mcp_save, reduce_lr],
              shuffle=True,
              verbose=1)

In [ ]:
print("Test Accuracy: {}%".format(acc*100))

In [ ]:
WIDTH=50
HIGHT=150
CHANNEL=3
 
def create_validation_data(DataDir):
    validation_data = []
    for category in CATEGORIES:
        path = os.path.join(DataDir, category) # path to each class dir
        test_class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = image.load_img(os.path.join(path,img),target_size=(HIGHT,WIDTH,CHANNEL))
                validation_data.append([img_array,test_class_num])
            except :
                pass
    return validation_data
 
valdiation_data=create_validation_data(DATADIR)


In [ ]:
desired_batch_size=7
test_dir=DATADIR
test_datagen = ImageDataGenerator(rescale=1./255)
 
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 50),
        color_mode="rgb",
        shuffle = False,
        class_mode='binary',
        batch_size=desired_batch_size)
 
filenames = test_generator.filenames
nb_samples = len(filenames)
 
predict = model_final.predict_generator(test_generator,steps = 
                                   np.ceil(nb_samples/desired_batch_size))

In [ ]:
## 10 epoch
def validation(test_dir):
    desired_batch_size = 7
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 50),
        color_mode="rgb",
        shuffle=False,
        class_mode='binary',
        batch_size=desired_batch_size)

    filenames = test_generator.filenames
    nb_samples = len(filenames)

    predict = model_final.predict_generator(test_generator, steps=
    np.ceil(nb_samples / desired_batch_size))

    correct = 0

    category = ["n", "p"]
    True_posetives = 0
    True_negatives = 0
    False_posetives = 0
    False_negatives = 0

    valdiation_data = create_validation_data(test_dir)

    for i in range(len(predict)):

        prediction = category[int(round(predict[i][0]))]
        label = valdiation_data[i][1]
        img_array = valdiation_data[i][0]

        if prediction != category[label]:
            plt.title('prediction:{0}   label:{1}'.format(prediction, category[label]))
            plt.imshow(img_array, cmap=plt.get_cmap('gray'))
            plt.show()
        if prediction == "p" and category[label] == "p":
            correct += 1
            True_posetives += 1
        elif prediction == "n" and category[label] == "n":
            True_negatives += 1
            correct += 1
        elif prediction == "p" and category[label] == "n":
            False_posetives += 1
        elif prediction == "n" and category[label] == "p":
            False_negatives += 1

    total_number_of_samples = len(predict)

    ##acc = (True_posetives+False_negatives)/total_number_of_samples

    acc = correct / total_number_of_samples

    print("Total number of samples ", total_number_of_samples)
    print("True_posetives ", True_posetives)
    print("True_negatives ", True_negatives)
    print("False_posetive ", False_posetives)
    print("False_negative ", False_negatives)

    print("accuracy ", acc)


validation('test_4')

In [ ]:
validation('test_3')

In [ ]:
validation('these as test')